# リアルタイム推論サービスの作成

このコースのトレーニングと機械学習モデルの登録には多くの時間を費やしました。次に、クライアントを使用して新しいデータから予測を取得できるリアルタイムサービスとしてモデルを展開します。

## ワークスペースに接続する

最初に行う必要があるのは、Azure ML SDKを使用してワークスペースに接続することです。

> **Note**: 前の演習を完了してから、Azureサブスクリプションとの認証済みセッションの有効期限が切れた場合、再認証するように求められます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## モデルのトレーニングと登録

それでは、モデルをトレーニングして登録しましょう。

In [ ]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace = ws, name = "diabetes-training")
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('data/diabetes.csv')

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model
model_file = 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# Complete the run
run.complete()

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

print('Model trained and registered.')

## モデルをWebサービスとしてデプロイする

糖尿病の可能性に基づいて患者を分類する機械学習モデルをトレーニングして登録しました。このモデルは、リスクがあると考えられる患者のみが糖尿病の臨床検査を受ける必要がある医師の手術などの生産環境で使用できます。このシナリオをサポートするには、モデルをWebサービスとしてデプロイします。

最初に、ワークスペースに登録したモデルを決定しましょう。

In [ ]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

それでは、デプロイしたいモデルを取得しましょう。デフォルトでは、モデル名を指定すると、最新バージョンが返されます。

In [ ]:
model = ws.models['diabetes_model']
print(model.name, 'version', model.version)

このモデルをホストするWebサービスを作成します。これには、いくつかのコードと構成ファイルが必要です。それらのフォルダを作成しましょう。

In [ ]:
import os

folder_name = 'diabetes_service'

# Create a folder for the web service files
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

print(folder_name, 'folder created.')

モデルをデプロイするWebサービスには、入力データをロードし、ワークスペースからモデルを取得し、予測を生成して返すためのPythonコードが必要です。このコードを、Webサービスにデプロイされる* entry script *に保存します:

In [ ]:
%%writefile $folder_name/score_diabetes.py
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = Model.get_model_path('diabetes_model')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

Webサービスはコンテナでホストされ、コンテナは初期化されたときに必要なPython依存関係をインストールする必要があります。この場合、スコア付けコードには**scikit-learn**が必要なので、コンテナホストにこれを環境にインストールするように指示する.ymlファイルを作成します。

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_conda_package('scikit-learn')

# Save the environment config as a .yml file
env_file = folder_name + "/diabetes_env.yml"
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())

これでデプロイする準備ができました。 **diabetes-service**という名前のサービスをコンテナーにデプロイします。展開プロセスには次の手順が含まれます:

1. モデルをロードして使用するために必要なスコアリングおよび環境ファイルを含む推論構成を定義します。
2. サービスがホストされる実行環境を定義するデプロイメント構成を定義します。この場合、Azure Container Instance。
3. モデルをWebサービスとしてデプロイします。
4. デプロイされたサービスのステータスを確認します。

> **詳しくは**: モデルの展開およびターゲット実行環境のオプションの詳細については、[ドキュメント](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where)を参照してください。

展開では、最初にコンテナイメージを作成するプロセスを実行し、次にイメージに基づいてWebサービスを作成するプロセスを実行するため、時間がかかります。展開が正常に完了すると、**Healthy**のステータスが表示されます。

In [ ]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   source_directory = folder_name,
                                   entry_script="score_diabetes.py",
                                   conda_file="diabetes_env.yml")

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "diabetes-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

うまくいけば、デプロイが成功し、**Healthy**のステータスを確認できるようになります。そうでない場合は、次のコードを使用してステータスを確認し、トラブルシューティングに役立つサービスログを取得できます。

In [ ]:
print(service.state)
print(service.get_logs())

# 変更を行って再デプロイする必要がある場合は、次のコードを使用して異常なサービスを削除する必要がある場合があります。
#service.delete()

[Azure ML Studio](https://ml.azure.com)のワークスペースを見て、ワークスペースにデプロイされたサービスを示す**エンドポイント**ページを表示します。

次のコードを実行して、ワークスペース内のWebサービスの名前を取得することもできます:

In [ ]:
for webservice_name in ws.webservices:
    print(webservice_name)

## Webサービスを使用する

サービスがデプロイされたら、クライアントアプリケーションからサービスを利用できるようになりました。

In [ ]:
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22]]
print ('Patient: {}'.format(x_new[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print(predicted_classes[0])

また、複数の患者の観察結果をサービスに送信し、それぞれの観察結果を取得することもできます。

In [ ]:
import json

# This time our input is an array of two feature arrays
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array or arrays to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data
predictions = service.run(input_data = input_json)

# Get the predicted classes.
predicted_classes = json.loads(predictions)
   
for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

上記のコードは、Azure ML SDKを使用してコンテナー化されたWebサービスに接続し、それを使用して糖尿病分類モデルから予測を生成します。実稼働環境では、Azure ML SDKを使用せず、Webサービスに対してHTTPリクエストを行うだけのビジネスアプリケーションがモデルを使用する可能性があります。

これらのアプリケーションがリクエストを送信する必要があるURLを決定しましょう:

In [ ]:
endpoint = service.scoring_uri
print(endpoint)

エンドポイントURIがわかったので、アプリケーションは単純にHTTPリクエストを行い、患者データをJSON（またはバイナリ）形式で送信し、予測されたクラスを受け取ることができます。

In [ ]:
import requests
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

認証を必要としないAzure Container Instance（ACI）サービスとしてWebサービスをデプロイしました。これは開発とテストには適していますが、運用環境では、Azure Kubernetes Service（AKS）クラスターへのデプロイと認証の有効化を検討する必要があります。これには、REST要求に**Authorization**ヘッダーを含める必要があります。

## サービスを削除する

不要になったサービスは、不要な料金が発生しないように削除する必要があります。

In [ ]:
service.delete()

モデルをサービスとして公開する方法の詳細については、[ドキュメント]（https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where）を参照してください。